In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/PHShome/bg615/Anaphylaxis_pipeline/fe5_cTAKES/fe_feature_detail_table_anaphylaxis.csv")
df

In [ ]:
# Remove the ".txt" suffix of each ProviderID
df["ProviderID"] = df["ProviderID"].str.replace(".txt", "", regex=False)
df

In [ ]:
df["Feature_Status"].value_counts()

In [ ]:
# Change all non-A values to U
df.loc[~df["Feature_Status"].isin(["A"]), "Feature_Status"] = "U"
df["Feature_Status"].value_counts()

In [ ]:
# Group the rows and aggregate the Feature_Status values of different NoteIDs by the order
order = {"A": 5, "N": 4, "H": 3, "X": 2, "U": 1}
df_grouped_status = df.groupby(["PatID", "EncounterID", "FeatureID", "Feature", "FE_CodeType", "Confidence"]).agg({"Feature_Status": lambda x: max(x, key=lambda y: order[y])}).reset_index()
df_grouped_status

In [ ]:
# Group the rows and aggregate the Feature_dt and ProviderID of different NoteIDs by the earliest date
df_grouped_date = df.groupby(["PatID", "EncounterID", "FeatureID", "Feature", "FE_CodeType", "Confidence"]).agg({"Feature_dt": "min", "ProviderID": "first"}).reset_index()
df_grouped_date

In [ ]:
# Merge the two DataFrames
df = pd.merge(df_grouped_date, df_grouped_status, on=["PatID", "EncounterID", "FeatureID", "Feature", "FE_CodeType", "Confidence"], how="inner")
df

In [ ]:
# Ensure that the columns are in the order of the original table
df = df[["PatID", "EncounterID", "FeatureID", "Feature_dt", "Feature", "FE_CodeType", "ProviderID", "Confidence", "Feature_Status"]]
df

In [ ]:
# Sort the rows by PatID and EncounterID
df.sort_values(by=["PatID", "EncounterID"], inplace=True)
df

In [ ]:
df.reset_index(drop=True)

In [ ]:
df["Feature_Status"].value_counts()

In [ ]:
df_filtered = df[df["Feature_Status"] == "A"]
df_filtered

In [ ]:
df_filtered["PatID"].nunique()

In [ ]:
df_filtered[['PatID', 'EncounterID']].drop_duplicates()

In [ ]:
df.to_csv("/PHShome/bg615/Anaphylaxis_pipeline/fe5_cTAKES/fe_feature_table_anaphylaxis.csv", index=False)

In [ ]:
df["PatID"].nunique()

In [ ]:
df_U = df[df["Feature_Status"] == "U"]
df_U

In [ ]:
df_U["PatID"].nunique()